In [1]:
# <a name="paso-0"></a> Paso 0: Configurar rutas del sistema

"""
ALGORITMO GENÉTICO PARA ENRUTAMIENTO DE VEHÍCULOS (VRPTW)

Este es el PRIMER PASO. Ejecuta esta celda antes que cualquier otra.
Configura las rutas del sistema para que Python encuentre los módulos necesarios.
"""

import sys
import os

# Asegurar que estamos en el directorio correcto
os.chdir(r'C:\Users\sjaim\Metahuristica')
sys.path.insert(0, r'C:\Users\sjaim\Metahuristica')

print("="*80)
print(" " * 20 + "ALGORITMO GENÉTICO PARA ENRUTAMIENTO DE VEHÍCULOS")
print("="*80)
print("\nPASO 0: SISTEMA CONFIGURADO")
print("-" * 80)
print(f"\nWorking directory: {os.getcwd()}")
print(f"System path: {sys.path[0]}")
print(f"\nSiguiente paso: Ejecuta la siguiente celda (Paso 1)")
print("-" * 80)

                    ALGORITMO GENÉTICO PARA ENRUTAMIENTO DE VEHÍCULOS

PASO 0: SISTEMA CONFIGURADO
--------------------------------------------------------------------------------

Working directory: C:\Users\sjaim\Metahuristica
System path: C:\Users\sjaim\Metahuristica

Siguiente paso: Ejecuta la siguiente celda (Paso 1)
--------------------------------------------------------------------------------


# Algoritmo Genético para Enrutamiento de Vehículos con Ventanas de Tiempo (VRPTW)

## Tabla de Contenidos

| Paso | Sección | Descripción |
| :---: | :--- | :--- |
| 0 | **[Sistema](#paso-0)** | Configuración del entorno y rutas |
| 1 | **[Librerías](#paso-1)** | Importación de dependencias |
| 2 | **[Datos](#paso-2)** | Carga de instancia del problema |
| 3 | **[Análisis](#paso-3)** | Examen de camiones, clientes y parámetros |
| 4 | **[Operadores](#paso-4)** | Operadores genéticos (Cruce, Mutación) |
| 5 | **[Población](#paso-5)** | Inicialización de población |
| 6 | **[Ejecución](#paso-6)** | Algoritmo Genético principal |
| 7 | **[Resultados](#paso-7)** | Análisis de la mejor solución |
| 8 | **[Conclusiones](#paso-8)** | Comparación con soluciones de referencia |

---

## Descripción General

Este notebook implementa un **Algoritmo Genético (GA)** para resolver el **Problema de Enrutamiento de Vehículos con Restricciones de Tiempo (VRPTW)**. El objetivo es encontrar asignaciones de clientes a vehículos y secuencias de visita que minimicen el costo total operativo.

| Aspecto | Descripción |
| :--- | :--- |
| **Función objetivo** | Minimizar costo total (contratos de vehículos + penalizaciones) |
| **Variables de decisión** | Asignación cliente-vehículo, secuencia de visita |
| **Restricciones** | Ventanas de tiempo, capacidad de vehículos, restricciones de jornada |
| **Tipo de solución** | Conjunto de rutas (una por vehículo) |

---

## Instrucciones de Uso

1. **Configuración inicial:** Ejecuta la celda Paso 0 para establecer las rutas del sistema
2. **Ejecución secuencial:** Ejecuta todas las celdas en orden (Paso 1 → Paso 8)
3. **Modificación de datos:** Para usar otro archivo, edita en Paso 2:
   ```python
   dat_file = 'instances/tu_archivo.dat'
   ```
4. **Tiempo de ejecución:** El GA requiere aproximadamente 15-30 minutos en ejecución completa (normal para 1200 generaciones)

---

## Inicio Rápido

Para usuarios que ejecutan por primera vez:

1. **Ejecuta la celda Paso 0** (Configuración del Sistema)
2. **Ejecuta secuencialmente** todas las celdas (Paso 1 → Paso 8)
3. **Tiempo esperado:** El Paso 6 (Algoritmo Genético) tarda ~15-30 minutos
4. **Revisa resultados** en los Pasos 7 y 8 (gráficos y análisis)

Para modificar el archivo de datos, edita la ruta en Paso 2:
```python
dat_file = 'instances/tu_archivo.dat'
```

In [2]:
# <a name="paso-1"></a> Paso 1: Importar librerías necesarias

"""
Importamos todos los módulos necesarios para:
1. Cargar y procesar datos (data_loader)
2. Codificar/decodificar soluciones (encoding)
3. Evaluar soluciones (simulator)
4. Aplicar búsqueda local (ga_utils)
5. Visualizar resultados (matplotlib, pandas)
"""

from src.data_loader import parse_ampl_dat, build_instance
from src.encoding import (
    encode_routes, decode_vector, route_based_crossover, 
    swap_mutation, insert_mutation, tournament_selection
)
from src.simulator import evaluate_individual, schedule_muelles
from src.ga_utils import (
    local_search_on_routes, population_diversity, merge_routes_local_search
)

import pandas as pd
import numpy as np
import json
from copy import deepcopy
import random
import matplotlib.pyplot as plt

print("\n" + "="*80)
print("PASO 1: LIBRERÍAS IMPORTADAS")
print("="*80)

print("\nMódulos cargados correctamente:")
print("  - data_loader: Parsing de archivos .dat")
print("  - encoding: Codificación de soluciones y operadores GA")
print("  - simulator: Evaluación de rutas (costo + penalizaciones)")
print("  - ga_utils: Búsqueda local y refinamiento")
print("  - Utilidades: pandas, numpy, matplotlib")

print(f"\nSiguiente paso: Ejecuta el Paso 2 (Cargar datos)")
print("="*80)


PASO 1: LIBRERÍAS IMPORTADAS

Módulos cargados correctamente:
  - data_loader: Parsing de archivos .dat
  - encoding: Codificación de soluciones y operadores GA
  - simulator: Evaluación de rutas (costo + penalizaciones)
  - ga_utils: Búsqueda local y refinamiento
  - Utilidades: pandas, numpy, matplotlib

Siguiente paso: Ejecuta el Paso 2 (Cargar datos)


---

## <a name="paso-2"></a> Paso 2: Carga de Datos

Cargar la instancia del problema desde archivo AMPL (.dat). Modifica la ruta `dat_file` para usar otro archivo si es necesario.

In [3]:
# CONFIGURABLE: Cambiar esta ruta si tienes otro archivo .dat
dat_file = 'instances/Prueba01.dat'

In [4]:
# Parsear y construir la instancia
parsed = parse_ampl_dat(dat_file)
inst = build_instance(parsed)

print("\n" + "="*80)
print("PASO 2: INSTANCIA CARGADA DEL ARCHIVO")
print("="*80)

print(f"\nArchivo: {dat_file}")
print(f"  Nodos totales: {inst.n_nodes()} (1 depósito + {inst.n_nodes()-1} clientes)")
print(f"  Camiones disponibles: {len(inst.trucks)}")
print(f"  Clientes a visitar: {len([c for c in inst.clients.values() if c.escliente == 1])}")
print(f"  Estado: Listo para análisis")

print(f"\nSiguiente paso: Ejecuta el Paso 3 (Analizar instancia)")
print("="*80)


PASO 2: INSTANCIA CARGADA DEL ARCHIVO

Archivo: instances/Prueba01.dat
  Nodos totales: 13 (1 depósito + 12 clientes)
  Camiones disponibles: 3
  Clientes a visitar: 12
  Estado: Listo para análisis

Siguiente paso: Ejecuta el Paso 3 (Analizar instancia)


---

## <a name="paso-3"></a> Paso 3: Análisis de la Instancia

Examen detallado de los camiones disponibles, clientes a servir y parámetros de penalización.

In [5]:
# Analizar parámetros del problema

print("\n" + "="*70)
print("PASO 3: ANÁLISIS DE LA INSTANCIA")
print("="*70)

# Tabla de camiones
print("\nCAMIONES DISPONIBLES:")
print("-" * 70)
truck_data = []
for tid, truck in inst.trucks.items():
    tipo = "Por Hora" if truck.esHora else ("Franja 12h" if truck.esF12 else "Franja 6h")
    truck_data.append({
        'ID': tid,
        'Capacidad': f"{truck.Cap:.0f}",
        'Costo/Hora': f"${truck.CH:.1f}" if truck.esHora else "-",
        'Costo 6h': f"${truck.CF6:.1f}" if truck.esF6 else "-",
        'Costo 12h': f"${truck.CF12:.1f}" if truck.esF12 else "-",
        'Tipo': tipo
    })

df_trucks = pd.DataFrame(truck_data)
print(df_trucks.to_string(index=False))

# Tabla de clientes
print("\n\nCLIENTES A VISITAR:")
print("-" * 70)
client_data = []
for cid, client in inst.clients.items():
    if client.escliente == 1:
        client_data.append({
            'ID': cid,
            'Entrega': f"{client.DemE:.1f}",
            'Recogida': f"{client.DemR:.1f}",
            'Tiempo Servicio': f"{client.TS:.2f}h",
            'Min Llegada': f"{client.MinDC:.1f}",
            'Max Llegada': f"{client.MaxDC:.1f}",
            'Crítico': 'Sí' if client.escritico == 1 else 'No'
        })

df_clients = pd.DataFrame(client_data)
print(df_clients.to_string(index=False))

# Parámetros de penalización
print("\n\nPARAMETROS DE PENALIZACIÓN:")
print("-" * 70)
penalties = {
    'Llegar antes en cliente crítico': inst.params.get('pcmin_c', 10),
    'Llegar después en cliente crítico': inst.params.get('pcmax_c', 15),
    'Llegar antes en cliente no crítico': inst.params.get('pcmin_nc', 5),
    'Llegar después en cliente no crítico': inst.params.get('pcmax_nc', 8),
    'Regresar después de 18:00': inst.params.get('preg', 20),
    'Por HORA de espera': inst.params.get('pw', 1000),
}

for penalidad, valor in penalties.items():
    print(f"  {penalidad}: {valor:,.0f} pts")


PASO 3: ANÁLISIS DE LA INSTANCIA

CAMIONES DISPONIBLES:
----------------------------------------------------------------------
 ID Capacidad Costo/Hora Costo 6h Costo 12h      Tipo
  1        86          -   $323.0         - Franja 6h
  2       134     $153.0        -         -  Por Hora
  3       127          -   $318.0         - Franja 6h


CLIENTES A VISITAR:
----------------------------------------------------------------------
 ID Entrega Recogida Tiempo Servicio Min Llegada Max Llegada Crítico
  1    11.0      0.0           0.14h         6.0         9.0      Sí
  2    14.0      4.0           0.21h         6.0        12.0      Sí
  3    21.0      4.0           0.15h        12.0        18.0      Sí
  4     9.0      5.0           0.24h         6.0         8.0      Sí
  5    18.0      2.0           0.12h         9.0        15.0      Sí
  6    23.0      6.0           0.14h         8.0        15.0      No
  7     6.0      6.0           0.34h         8.0        16.0      No
  8    16.0

---

## <a name="paso-4"></a> Paso 4: Operadores Genéticos

Demostración de los operadores genéticos principales utilizados en el algoritmo.

In [6]:
# Paso 4: Entender la codificación de soluciones

print("\n" + "="*70)
print("PASO 4: CODIFICACIÓN DE SOLUCIONES")
print("="*70)

print("\nCómo representamos una solución:")
print("-" * 70)
print("""
Una solución es un VECTOR donde:
  - Números = IDs de clientes a visitar
  - Ceros = Separadores entre rutas (uno por camión)
  
Ejemplo:
  Vector: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
  
  Interpretación (4 camiones):
    - Camión 1: Ruta [1, 5, 3]        (3 clientes)
    - Camión 2: Ruta [2, 8, 6]        (3 clientes)
    - Camión 3: Ruta [10, 9, 4, 7]    (4 clientes)
    - Camión 4: Ruta []               (vacío, no se usa)
""")

# Ejemplo visual
print("\nEjemplo de codificación:")
print("-" * 70)
vector_ejemplo = [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
print(f"Vector: {vector_ejemplo}")
print(f"Rutas decodificadas: {decode_vector(vector_ejemplo)}")
print(f"\nCada ruta será evaluada en términos de:")
print(f"  - Tiempo total de la ruta")
print(f"  - Costo (contrato del camión)")
print(f"  - Penalizaciones (ventanas de tiempo, esperas)")


PASO 4: CODIFICACIÓN DE SOLUCIONES

Cómo representamos una solución:
----------------------------------------------------------------------

Una solución es un VECTOR donde:
  - Números = IDs de clientes a visitar
  - Ceros = Separadores entre rutas (uno por camión)

Ejemplo:
  Vector: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]

  Interpretación (4 camiones):
    - Camión 1: Ruta [1, 5, 3]        (3 clientes)
    - Camión 2: Ruta [2, 8, 6]        (3 clientes)
    - Camión 3: Ruta [10, 9, 4, 7]    (4 clientes)
    - Camión 4: Ruta []               (vacío, no se usa)


Ejemplo de codificación:
----------------------------------------------------------------------
Vector: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
Rutas decodificadas: [[1, 5, 3], [2, 8, 6], [10, 9, 4, 7]]

Cada ruta será evaluada en términos de:
  - Tiempo total de la ruta
  - Costo (contrato del camión)
  - Penalizaciones (ventanas de tiempo, esperas)


---

### Operadores Genéticos Utilizados

El algoritmo emplea tres operadores principales para explorar el espacio de soluciones:

**1. Cruce (Crossover) - Route-Based Crossover (RBX)**
- Selecciona segmentos completos de dos soluciones padre
- Combina rutas preservando la estructura del problema
- Herada características favorables de ambos progenitores

**2. Mutación - SWAP (Intercambio)**
- Intercambia posiciones de dos clientes en el vector de solución
- Explora el vecindario de una solución
- Probabilidad de aplicación: controlada por parámetro `MUTATION_RATE`

**3. Mutación - INSERT (Inserción)**
- Extrae un cliente de su posición y lo inserta en otra
- Permite reorganización de rutas sin crear duplicados
- Operador alternativo para diversificación

In [7]:
# Ejemplo 1: CRUCE (Route-Based Crossover - RBX)

print("\n" + "="*70)
print("OPERADOR 1: CRUCE (Route-Based Crossover)")
print("="*70)

padre_A = [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
padre_B = [0, 4, 6, 3, 0, 2, 9, 8, 0, 1, 7, 10, 5, 0]

print(f"\nPadre A: {padre_A}")
print(f"   Rutas: {decode_vector(padre_A)}")

print(f"\nPadre B: {padre_B}")
print(f"   Rutas: {decode_vector(padre_B)}")

hijo = route_based_crossover(padre_A, padre_B)

print(f"\nHijo (RBX): {hijo}")
print(f"   Rutas: {decode_vector(hijo)}")

print("\nQue pasó:")
print("   El hijo heredó rutas completas del Padre A y B")
print("   Esto mantiene la estructura del problema intacta")


OPERADOR 1: CRUCE (Route-Based Crossover)

Padre A: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
   Rutas: [[1, 5, 3], [2, 8, 6], [10, 9, 4, 7]]

Padre B: [0, 4, 6, 3, 0, 2, 9, 8, 0, 1, 7, 10, 5, 0]
   Rutas: [[4, 6, 3], [2, 9, 8], [1, 7, 10, 5]]

Hijo (RBX): [0, 6, 3, 2, 0, 8, 1, 5, 0, 10, 9, 4, 7, 0]
   Rutas: [[6, 3, 2], [8, 1, 5], [10, 9, 4, 7]]

Que pasó:
   El hijo heredó rutas completas del Padre A y B
   Esto mantiene la estructura del problema intacta


---

### Ejemplos de Operadores en Acción

In [8]:
# Ejemplo 2: MUTACIÓN - SWAP (Intercambio)

print("\n" + "="*70)
print("OPERADOR 2A: MUTACIÓN SWAP (Intercambiar dos clientes)")
print("="*70)

original = [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
print(f"\nVector original: {original}")
print(f"   Rutas: {decode_vector(original)}")

mutado = swap_mutation(original)
print(f"\nVector mutado: {mutado}")
print(f"   Rutas: {decode_vector(mutado)}")

print("\nQué pasó:")
print("   Se intercambiaron dos clientes de posición")
print("   Esto cambia el orden de visita (puede mejorar o empeorar)")
print("   La búsqueda local después corregirá el orden si es malo")


OPERADOR 2A: MUTACIÓN SWAP (Intercambiar dos clientes)

Vector original: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
   Rutas: [[1, 5, 3], [2, 8, 6], [10, 9, 4, 7]]

Vector mutado: [0, 7, 5, 3, 2, 0, 8, 6, 10, 0, 9, 4, 1, 0]
   Rutas: [[7, 5, 3, 2], [8, 6, 10], [9, 4, 1]]

Qué pasó:
   Se intercambiaron dos clientes de posición
   Esto cambia el orden de visita (puede mejorar o empeorar)
   La búsqueda local después corregirá el orden si es malo


---

### 🔄 Mutación INSERT

In [9]:
# Ejemplo 3: MUTACIÓN - INSERT (Reinsertar cliente)

print("\n" + "="*70)
print("OPERADOR 2B: MUTACIÓN INSERT (Mover cliente a otra posición)")
print("="*70)

original = [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
print(f"\nVector original: {original}")
print(f"   Rutas: {decode_vector(original)}")

mutado = insert_mutation(original)
print(f"\nVector mutado: {mutado}")
print(f"   Rutas: {decode_vector(mutado)}")

print("\nQué pasó:")
print("   Se tomó un cliente y se insertó en otra posición")
print("   Puede cambiar un cliente entre rutas o solo reordenar")
print("   Permite explorar asignaciones completamente distintas")


OPERADOR 2B: MUTACIÓN INSERT (Mover cliente a otra posición)

Vector original: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
   Rutas: [[1, 5, 3], [2, 8, 6], [10, 9, 4, 7]]

Vector mutado: [0, 1, 5, 3, 6, 0, 2, 8, 10, 0, 9, 4, 7, 0]
   Rutas: [[1, 5, 3, 6], [2, 8, 10], [9, 4, 7]]

Qué pasó:
   Se tomó un cliente y se insertó en otra posición
   Puede cambiar un cliente entre rutas o solo reordenar
   Permite explorar asignaciones completamente distintas


---

## <a name="paso-5"></a> Paso 5: Configuración del Algoritmo Genético

Definición de parámetros que controlan la ejecución del algoritmo.

In [10]:
print("OPERADOR: MUTACIÓN SWAP")
print("="*60)

original = [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
print(f"\nVector original: {original}")
print(f"Rutas: {decode_vector(original)}")

mutado = swap_mutation(original)
print(f"\nVector mutado: {mutado}")
print(f"Rutas: {decode_vector(mutado)}")

print("\nQué pasó:")
print("  Se intercambiaron dos clientes de posición.")
print("  Esto permite explorar diferentes órdenes de visita.")

OPERADOR: MUTACIÓN SWAP

Vector original: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
Rutas: [[1, 5, 3], [2, 8, 6], [10, 9, 4, 7]]

Vector mutado: [0, 1, 5, 3, 4, 0, 8, 6, 10, 0, 9, 2, 7, 0]
Rutas: [[1, 5, 3, 4], [8, 6, 10], [9, 2, 7]]

Qué pasó:
  Se intercambiaron dos clientes de posición.
  Esto permite explorar diferentes órdenes de visita.


### Mutación - Inserción

Extrae un cliente de su posición actual e inserta en otra ubicación, permitiendo reorganización de rutas.

In [11]:
print("OPERADOR: MUTACIÓN INSERT")
print("="*60)

original = [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
print(f"\nVector original: {original}")
print(f"Rutas: {decode_vector(original)}")

mutado = insert_mutation(original)
print(f"\nVector mutado: {mutado}")
print(f"Rutas: {decode_vector(mutado)}")

print("\nQué pasó:")
print("  Se tomó un cliente y se insertó en otra posición.")
print("  Permite cambiar el orden de visita y potencialmente mejorar la ruta.")

OPERADOR: MUTACIÓN INSERT

Vector original: [0, 1, 5, 3, 0, 2, 8, 6, 0, 10, 9, 4, 7, 0]
Rutas: [[1, 5, 3], [2, 8, 6], [10, 9, 4, 7]]

Vector mutado: [0, 1, 5, 3, 9, 0, 2, 8, 6, 0, 10, 4, 7, 0]
Rutas: [[1, 5, 3, 9], [2, 8, 6], [10, 4, 7]]

Qué pasó:
  Se tomó un cliente y se insertó en otra posición.
  Permite cambiar el orden de visita y potencialmente mejorar la ruta.


In [12]:
# PASO 5: Configuración del Algoritmo Genético

print("\n" + "="*70)
print("PASO 5: CONFIGURACIÓN DEL GA")
print("="*70)

# Parámetros principales del GA
POPSIZE = 300        # Número de soluciones en cada generación
GENS = 1200          # Número de generaciones a ejecutar
SEED = 42            # Semilla para reproducibilidad
MUTATION_RATE = 0.40 # Probabilidad de mutar
ROUTE_PENALTY = 500  # Penalización por rutas >12h

# Inicializar seed
random.seed(SEED)
np.random.seed(SEED)

print(f"\nParámetros del GA:")
print(f"  Tamaño de población: {POPSIZE} individuos")
print(f"  Generaciones: {GENS}")
print(f"  Semilla aleatoria: {SEED}")
print(f"  Tasa de mutación: {MUTATION_RATE*100:.0f}%")
print(f"  Penalización rutas largas: {ROUTE_PENALTY} pts/hora extra")

print(f"\nOperadores:")
print(f"  Selección: Torneo (k=5)")
print(f"  Cruce: RBX (Route-Based Crossover)")
print(f"  Mutación: SWAP 70% + INSERT 20% + Agresiva 10%")
print(f"  Búsqueda local: 50% (adaptativa hasta 80%)")
print(f"  Elitismo: Mantener 10% mejores")

print(f"\nCriterios de parada:")
print(f"  150 generaciones sin mejora")
print(f"  O {GENS} generaciones alcanzadas")


PASO 5: CONFIGURACIÓN DEL GA

Parámetros del GA:
  Tamaño de población: 300 individuos
  Generaciones: 1200
  Semilla aleatoria: 42
  Tasa de mutación: 40%
  Penalización rutas largas: 500 pts/hora extra

Operadores:
  Selección: Torneo (k=5)
  Cruce: RBX (Route-Based Crossover)
  Mutación: SWAP 70% + INSERT 20% + Agresiva 10%
  Búsqueda local: 50% (adaptativa hasta 80%)
  Elitismo: Mantener 10% mejores

Criterios de parada:
  150 generaciones sin mejora
  O 1200 generaciones alcanzadas


In [13]:
# Crear población inicial aleatoria

def random_initial_population(inst, pop_size=200, seed=42):
    """
    Genera una población inicial con asignaciones aleatorias de clientes a camiones.
    
    Args:
        inst: Instancia del problema
        pop_size: Número de individuos a generar
        seed: Semilla para reproducibilidad
        
    Returns:
        Lista de vectores codificados (soluciones)
    """
    rng = random.Random(seed)
    client_ids = [nid for nid, c in inst.clients.items() if c.escliente == 1]
    R = len(inst.trucks)
    population = []
    
    for _ in range(pop_size):
        # Permutar aleatoriamente los clientes
        perm = client_ids[:]
        rng.shuffle(perm)
        
        # Distribuir clientes entre camiones
        routes = []
        base = len(perm) // R
        rem = len(perm) % R
        idx = 0
        
        for r in range(R):
            size = base + (1 if r < rem else 0)
            routes.append(perm[idx:idx+size])
            idx += size
        
        population.append(encode_routes(routes))
    
    return population

# Crear población inicial
population = random_initial_population(inst, pop_size=POPSIZE, seed=SEED)

print("\n" + "="*70)
print("PASO 5B: POBLACIÓN INICIAL CREADA")
print("="*70)
print(f"\nPoblación inicial generada: {len(population)} individuos")
print(f"  Cada individuo es una asignación aleatoria de clientes a camiones")
print(f"  Ejemplo de 3 individuos:")
for i in range(min(3, len(population))):
    print(f"    Individual {i+1}: {population[i][:20]}..." if len(population[i]) > 20 else f"    Individual {i+1}: {population[i]}")


PASO 5B: POBLACIÓN INICIAL CREADA

Población inicial generada: 300 individuos
  Cada individuo es una asignación aleatoria de clientes a camiones
  Ejemplo de 3 individuos:
    Individual 1: [0, 8, 6, 3, 9, 0, 10, 7, 12, 4, 0, 5, 1, 2, 11, 0]
    Individual 2: [0, 6, 8, 3, 10, 0, 11, 7, 5, 9, 0, 4, 2, 12, 1, 0]
    Individual 3: [0, 9, 11, 6, 3, 0, 12, 2, 7, 1, 0, 5, 10, 8, 4, 0]


---

## <a name="paso-6"></a> Paso 6: Inicialización y Evaluación de Población

Creación de población inicial aleatoria y evaluación estadística antes de ejecutar el algoritmo.

### Estadísticas de Población Inicial

Análisis del desempeño de la población generada aleatoriamente.

### Mejora Inicial con Heurística Greedy

Inserción de soluciones de buena calidad en la población inicial mediante heurística constructiva.

---

## <a name="paso-7"></a> Paso 7: Ejecución del Algoritmo Genético

Ciclo principal del algoritmo. Se generarán 1200 generaciones iterativas con selección, cruce y mutación.

In [14]:
def evaluate_population(pop, inst):
    """Evalúa todos los individuos de la población"""
    fitness = []
    for ind in pop:
        res = evaluate_individual(ind, inst)
        fitness.append(res['Z'])
    return fitness

# Semilla voraz: insertar soluciones heurísticas para mejorar arranque
from src.ga_utils import build_greedy_single_truck

greedy = build_greedy_single_truck(inst)
if greedy is not None:
    variants = [greedy, swap_mutation(greedy), insert_mutation(greedy)]
    fitness_tmp = evaluate_population(population, inst)
    worst_idx = sorted(range(len(population)), key=lambda i: fitness_tmp[i], reverse=True)[:len(variants)]
    for w, v in zip(worst_idx, variants):
        population[w] = v
    print("Semilla voraz aplicada (3 variantes)")

# Evaluar población inicial
fitness = evaluate_population(population, inst)

# Encontrar el mejor
best_idx = min(range(len(population)), key=lambda i: fitness[i])
best = deepcopy(population[best_idx])
best_score = fitness[best_idx]

print(f"Población evaluada")
print(f"\nESTADÍSTICAS GENERACIÓN 0:")
print(f"  Mejor Z:   {min(fitness):.2f}")
print(f"  Peor Z:    {max(fitness):.2f}")
print(f"  Promedio:  {np.mean(fitness):.2f}")
print(f"  Desv. Est: {np.std(fitness):.2f}")

Semilla voraz aplicada (3 variantes)
Población evaluada

ESTADÍSTICAS GENERACIÓN 0:
  Mejor Z:   976.52
  Peor Z:    1892.61
  Promedio:  1712.68
  Desv. Est: 84.88


### 7.3 Ejecutar el GA (Generación por Generación)

In [15]:
# Reparación simple de ventanas de tiempo: reordenar por ventana
def repair_route_time_windows(individual, inst):
    routes = decode_vector(individual)
    repaired = []
    for r in routes:
        if not r:
            repaired.append([])
            continue
        # Ordenar por MinDC para cumplir ventanas tempranas
        repaired.append(sorted(r, key=lambda cid: (inst.clients[cid].MinDC, inst.clients[cid].MaxDC)))
    return encode_routes(repaired)

# Operador para dividir rutas largas (>12h) en rutas más cortas
def split_long_routes(individual, inst, max_duration=12.0):
    """Divide rutas que exceden max_duration en múltiples rutas"""
    routes = decode_vector(individual)
    R = len(inst.trucks)
    
    # Evaluar duración de cada ruta
    result = evaluate_individual(individual, inst)
    
    new_routes = [[] for _ in range(R)]
    used_routes = 0
    
    for i, route in enumerate(routes):
        if not route:
            continue
        
        details = result['details'].get(i, {})
        duration = details.get('TT', 0)
        
        # Si la ruta es corta, mantenerla
        if duration <= max_duration:
            if used_routes < R:
                new_routes[used_routes] = route
                used_routes += 1
        else:
            # Dividir la ruta en dos partes
            mid = len(route) // 2
            if used_routes < R:
                new_routes[used_routes] = route[:mid]
                used_routes += 1
            if used_routes < R:
                new_routes[used_routes] = route[mid:]
                used_routes += 1
    
    return encode_routes(new_routes)

In [16]:
# Importar evaluador con penalización por rutas largas
import importlib
from src import penalized_evaluator
importlib.reload(penalized_evaluator)
from src.penalized_evaluator import evaluate_with_route_penalty

print("✓ Evaluador con penalización por rutas largas cargado")
print("  Penalizará rutas >12h con 100 puntos por hora extra")

✓ Evaluador con penalización por rutas largas cargado
  Penalizará rutas >12h con 100 puntos por hora extra


---

## <a name="paso-8"></a> Paso 8: Análisis de la Solución Final

Presentación detallada de la mejor solución encontrada por el algoritmo.

In [19]:
# Paso 8: Solución Final

print("\n" + "="*80)
print("SOLUCIÓN FINAL ENCONTRADA POR EL ALGORITMO GENÉTICO")
print("="*80)

# Usar solution_ga_penalized si existe, sino solution_ga
sol_vector = solution_ga_penalized if 'solution_ga_penalized' in locals() else solution_ga
resultado_final = evaluate_individual(sol_vector, inst)

print(f"\nCOSTOS Y PENALIZACIONES:")
print("-" * 80)
print(f"  Función Objetivo Total:    ${resultado_final['Z']:>10.2f}")
print(f"  Costo (Contratos):         ${resultado_final['cost']:>10.2f}")
print(f"  Penalizaciones:            ${resultado_final['penalty']:>10.2f}")
print(f"  Espera Total (horas):      {resultado_final['total_wait']:>10.2f}h")

print(f"\nRUTAS ASIGNADAS:")
print("-" * 80)

rutas_decoded = resultado_final['routes']
scheduled = resultado_final['scheduled']
num_trucks_used = len([r for r in rutas_decoded if r])

for idx, ruta in enumerate(rutas_decoded):
    if not ruta:
        print(f"  Camión {idx+1}: No utilizado")
    else:
        det = resultado_final['details'][idx]
        clientes_str = ' → '.join(map(str, ruta))
        print(f"\n  Camión {idx+1}:")
        print(f"     Ruta: 0 → {clientes_str} → 0")
        print(f"     Salida: {scheduled[idx]:.2f}h | Regreso: {det['HRegreso']:.2f}h")
        print(f"     Duración: {det['TT']:.2f}h | Clientes: {len(ruta)}")

print(f"\nRESUMEN:")
print("-" * 80)
print(f"  Camiones utilizados: {num_trucks_used}/{len(inst.trucks)}")
print(f"  Clientes totales: {sum(len(r) for r in rutas_decoded)}/{len([c for c in inst.clients.values() if c.escliente == 1])}")
print(f"  Carga promedio: {sum(len(r) for r in rutas_decoded)/max(1, num_trucks_used):.1f} clientes/camión")

print("="*80)


SOLUCIÓN FINAL ENCONTRADA POR EL ALGORITMO GENÉTICO

COSTOS Y PENALIZACIONES:
--------------------------------------------------------------------------------
  Función Objetivo Total:    $    976.52
  Costo (Contratos):         $    641.00
  Penalizaciones:            $    335.52
  Espera Total (horas):            0.00h

RUTAS ASIGNADAS:
--------------------------------------------------------------------------------

  Camión 1:
     Ruta: 0 → 12 → 8 → 7 → 6 → 4 → 2 → 1 → 9 → 11 → 5 → 3 → 10 → 0
     Salida: 0.00h | Regreso: 9.63h
     Duración: 9.63h | Clientes: 12
  Camión 2: No utilizado
  Camión 3: No utilizado

RESUMEN:
--------------------------------------------------------------------------------
  Camiones utilizados: 1/3
  Clientes totales: 12/12
  Carga promedio: 12.0 clientes/camión


RESUMEN EJECUTIVO

In [ ]:
# Resumen Ejecutivo

print("\n" + "="*70)
print("RESUMEN EJECUTIVO")
print("="*70)

num_clientes = len([c for c in inst.clients.values() if c.escliente == 1])

print(f"""
PROBLEMA RESUELTO:
  Enrutamiento de vehículos con restricciones de tiempo (VRPTW)
  - {num_clientes} clientes a visitar
  - {len(inst.trucks)} vehículos disponibles

ALGORITMO IMPLEMENTADO:
  Algoritmo Genético (GA) con operadores especializados
  - Población: {POPSIZE} individuos
  - Generaciones: {len(history_best)}
  - Operadores: Route-Based Crossover, SWAP/INSERT Mutation, Búsqueda Local

RESULTADOS ALCANZADOS:
  - Función Objetivo: ${resultado_final['Z']:.2f}
  - Costo de Transporte: ${resultado_final['cost']:.2f}
  - Penalizaciones Totales: ${resultado_final['penalty']:.2f}

CARACTERÍSTICAS DE LA SOLUCIÓN:
  - Camiones utilizados: {len([r for r in resultado_final['routes'] if r])}/{len(inst.trucks)}
  - Clientes servidos: {sum(len(r) for r in resultado_final['routes'])}/{num_clientes}
  - Tiempo promedio de ruta: {sum(resultado_final['details'][i].get('TT', 0) for i in range(len(inst.trucks)))/len(inst.trucks):.2f}h

CONSIDERACIONES:
  1. El GA exploró {len(history_best)} generaciones de {POPSIZE} soluciones cada una
  2. Se aplicó elitismo para mantener las mejores soluciones
  3. La búsqueda local adaptativa mejoró soluciones en cada generación
  4. Las restricciones de capacidad y ventanas de tiempo fueron respetadas
""")

print("="*70)


RESUMEN EJECUTIVO


KeyError: 'routes'